# Imports

In [1]:
from pynq.overlays.base import BaseOverlay
import time, multiprocessing, socket
base = BaseOverlay("base.bit")
btns = base.btns_gpio

# Buzzer

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

int write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
    return 0;
}

In [3]:
pin = 0

def buzz():
    write_gpio(pin, 1)
    time.sleep(1/2)
    write_gpio(pin, 0)
 
        
buzz()

# Server

In [21]:
def server(startTime):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    host = ''
    port = 8000
    sock.bind((host, port)) 
    sock.listen()
    print("The server is ready to receive")
    conn, addr = sock.accept()
    print('Connected by', addr)
    while True:
        data = conn.recv(1024).decode()
        print(data)
        if len(data) > 0:
            buzz()
        elif len(data) == 0:
            break
        time.sleep(1)
    sock.close()
    endTime = time.time()
    print('Server Runtime: {}'.format(endTime - startTime))

# Client

In [22]:
def client(startTime):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    host = '192.168.2.1'
    port = 8001
    print("Attempting Connnection")
    sock.connect((host, port))
    print("The client is ready to send messages")
    
    while True:
        if btns[1].read() != 0:
            data = "buzz"
            sock.send(data.encode())
        if btns[2].read() != 0:
            print("Closing the client socket...")
            sock.close()
            print("Done!")
            break
        time.sleep(.2)
    endTime = time.time()
    print('Client Runtime: {}'.format(endTime - startTime))
    
def start_client():
    print("The client is ready to start")
    while True:
        if btns[0].read() != 0:
            print("Starting client...")
            startTime = time.time()
            client(startTime)
        if btns[2].read() != 0:
            break      

# Multiprocessing

In [27]:
# Launch server
srvrStartTime = time.time()
srvr = multiprocessing.Process(target=server, args=(srvrStartTime,)) 
srvr.start() 

# Launch client
clnt = multiprocessing.Process(target=start_client, args=())
clnt.start() 

sName = srvr.name 
cName = clnt.name 

clnt.join() 
print('Client {} has finished'.format(cName))

srvr.join() # wait for the server to finish
print('Server {} has finished'.format(sName))

The server is ready to receive
The client is ready to start
Connected by ('192.168.2.1', 18878)
Starting client...
Attempting Connnection
The client is ready to send messages
buzz
buzz
buzz
buzzbuzz
buzzbuzz
buzz
buzzbuzzbuzz
buzzbuzzbuzzbuzzbuzz
buzz
buzz
buzzbuzzbuzzbuzzbuzz
buzzbuzzbuzz
buzz

Server Runtime: 417.73328375816345
Closing the client socket...
Done!
Client Runtime: 350.20819878578186
Client Process-27 has finished
Server Process-26 has finished
